### Import data

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
#Load the data
#name='path of file'
#with_info= provides a tuple containing info about version, features and # samples of the data set
#as_supervised = True, loads the data in a 2-tuple structure [input,target]
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

2024-03-31 22:17:32.534060: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/kyotun/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
#extracting data
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

### Preprocess data

In [10]:
#validation comes from train part, there is no specific part for validation
#train contains 60,000 examples
#test contains 10,000 examples

#define the number of validation samples
#but we don't know if this num is integer or float
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

In [11]:
#therefore, we convert the variable into a given data type
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [12]:
#store the num of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [13]:
#normally we'd like to scale the datain some way to make the result more numerically stable (inputs between 0-1)

#take an MNIST iamge and its label
#0-255
#we want float
#you can scale your data in other ways if you see fit.
#make sure that func. takes image and label
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [14]:
#scale the whole train dataset and store it in variable
scaled_train_and_validation_data = mnist_train.map(scale)

In [15]:
test_data = mnist_test.map(scale)

In [16]:
#what if every batch consist different procent of same element?
#that is why, we need to shuffle the data as random as possible, in order to get the best results
#therefore, we need buffer in order to put all data and shuffle in that buffer the all data at one time
#if buffer size >= num_sample, shuffling will happen at once (uniformly)
#if 1 < buffer size <= num_sample, we'll be optimizing the computational power of computer
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [18]:
#after that we can extract the exact train and validation sets
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)
#.skip() method skips the given parameter and take the datas what is left from (all-given parameter)

In [19]:
#we'll use minibatch GD -> Sample size / batch size = element_in_one_batch
#batch_size = 1 -> SGD
#batch_size = # samples GD(single batch)
#1<batch_size z # samples -> mini batch

BATCH_SIZE = 100

In [20]:
#.batch(SIZE) -> put the given data into the given sizes of batches
train_data = train_data.batch(BATCH_SIZE)
#update the weights every batch(in every 100 element)

In [21]:
#we need to reshape validation_data into batch format too but when we forward propagate we need just 1 batch cause
#we take all the datas at once, because we want exact values(not updating in every 100 element)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [22]:
#validation data must have the same shape and obj. properties as the train and test data
#remember that we did as_supervised -> loads the data in 2-Tuple structure [inputs,targets]
#therefore, we need extract and convert the validation inputs/targets too

validation_inputs, validation_targets = next(iter(validation_data))
#.iter() = creates an obj. which can be iterated one element at a time
#.next() = loads the next element of an iterable object

2024-03-31 22:27:54.350246: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-31 22:27:54.819712: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
